In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks/ESG/ESG-NEW
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ESG/ESG-NEW
bert_model.py  csv_gpt_label  main.ipynb  parse_pdf.py	pdf_dir  __pycache__


# Parse pdf (or xml)

In [3]:
!pip install pdfminer.six
from parse_pdf import file2csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.0 MB/s eta 0:00:00


## Parse pdf into csv using file2csv()

In [5]:
import os

PDF_DIR = 'pdf_dir'
NATIONS = 'american', 'taiwan'
OUTPUT_DIR = 'output'
PARSED_DIR = os.path.join(OUTPUT_DIR, 'parsed')

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
if not os.path.exists(PARSED_DIR):
    os.mkdir(PARSED_DIR)

In [6]:
%%time
for nation in NATIONS:
    pdf_dir = os.path.join(PDF_DIR, nation)
    csv_output_dir = os.path.join(PARSED_DIR, nation)
    if not os.path.exists(csv_output_dir):
        os.mkdir(csv_output_dir)
    for pdf_name in os.listdir(pdf_dir):
        pdf_path = os.path.join(pdf_dir, pdf_name)
        print(pdf_path)
        file2csv(pdf_path, csv_output_dir)

pdf_dir/american/ikea_sustainability_report_fy22_57c0217c71.pdf
pdf_dir/american/Flexsteel 2022-ESG-Report.pdf
pdf_dir/taiwan/台灣-Mediatek(聯發科).pdf
pdf_dir/taiwan/台灣-Cathay Financial.pdf
CPU times: user 1min 11s, sys: 221 ms, total: 1min 11s
Wall time: 1min 20s


# EsgBert (remember to use GPU)

## Vanilla nbroad/ESG-BERT

In [7]:
from bert_model import EsgBertPredict

LABLED_DIR = os.path.join(OUTPUT_DIR, 'labled')
if not os.path.exists(LABLED_DIR):
    os.mkdir(LABLED_DIR)

esg_bert = EsgBertPredict()
for nation in NATIONS:
    csv_dir = os.path.join(PARSED_DIR, nation)
    output_dir = os.path.join(LABLED_DIR, nation)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for csv_name in os.listdir(csv_dir):
        csv_path = os.path.join(csv_dir, csv_name)
        print(csv_path)
        esg_bert.test(csv_path, output_dir)

cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

output/parsed/american/ikea_sustainability_report_fy22_57c0217c71.csv


...

100%|██████████| 21/21 [00:01<00:00, 13.83it/s]


output/parsed/taiwan/台灣-Mediatek(聯發科).csv


100%|██████████| 78/78 [00:10<00:00,  7.51it/s]


output/parsed/taiwan/台灣-Cathay Financial.csv


100%|██████████| 96/96 [00:08<00:00, 11.96it/s]


## Finetune with a csv

In [8]:
esg_bert.train_data('csv_gpt_label/gpt_american.csv')
esg_bert.training()

100%|██████████| 93/93 [00:29<00:00,  3.17it/s]


Epoch: 0 Train Loss 531.4385


100%|██████████| 24/24 [00:02<00:00,  9.05it/s]

Epoch: 0 Valid Loss 124.1720


In [9]:
FINETUNE_DIR = os.path.join(OUTPUT_DIR, 'finetune_labled')
if not os.path.exists(FINETUNE_DIR):
    os.mkdir(FINETUNE_DIR)

for nation in NATIONS:
    csv_dir = os.path.join(PARSED_DIR, nation)
    output_dir = os.path.join(FINETUNE_DIR, nation)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for csv_name in os.listdir(csv_dir):
        csv_path = os.path.join(csv_dir, csv_name)
        print(csv_path)
        esg_bert.test(csv_path, output_dir)

output/parsed/american/ikea_sustainability_report_fy22_57c0217c71.csv


100%|██████████| 83/83 [00:06<00:00, 13.63it/s]


output/parsed/american/Flexsteel 2022-ESG-Report.csv


100%|██████████| 21/21 [00:01<00:00, 12.48it/s]


output/parsed/taiwan/台灣-Mediatek(聯發科).csv


100%|██████████| 78/78 [00:11<00:00,  6.60it/s]


output/parsed/taiwan/台灣-Cathay Financial.csv


100%|██████████| 96/96 [00:09<00:00, 10.34it/s]
